In [ ]:
# using Pkg
# Pkg.DEFAULT_IO[] = stdout;
# Pkg.status()

using LinearAlgebra
using Random
using Juqbox
using JSON
using Dates
using DelimitedFiles
using Distributions

include("pulsegen/generate_unitary.jl");
include("pulsegen/juqbox_environment_setup.jl");
include("pulsegen/io.jl");
include("pulsegen/wrapper.jl");

In [2]:
function get_data_path(
        prefix::String,
        config_path::String,
        randomseed::Integer=1234,
        use_timestamp::Bool=true)
    data_path = prefix;
    data_path *= "_configfile_" * config_path;
    data_path *= "_randomseed_" * lpad(randomseed,4,"0");
    if use_timestamp
        data_path *= "_" * string(now());
    end
    data_path *= ".csv";
    return data_path;
end

get_data_path (generic function with 3 methods)

In [3]:
# Common param
gammamin = -pi
gammamax = pi
ngamma = 16384
config_path = "config_template_D1_10_Pmin_200.json";
#config_path = "config_template_D1_6_Pmin_200.json";
#config_path = "config_template_1_guardlevel.json";
#config_path = "config_template.json";
initial_amplitude_relative_maximum = 0.01;

# Train data param
prefix_train = "data_pulse/ry_test"
full_range_train = range(gammamin, stop=gammamax,length=ngamma)

# Test data param
prefix_test = "data_pulse/ry_random"
full_range_test = rand(Uniform(-pi,pi),1,ngamma)

1×16384 Matrix{Float64}:
 -0.651536  2.78704  -2.56468  0.059777  …  -2.97659  2.16698  -0.737598

Generate train data:

In [4]:
@time for randomseed in 1:1

    data_path = get_data_path(prefix_train, config_path, randomseed, false);
    open(data_path, "w") do io
        write(io, "");
    end;

    for gamma in full_range_train
        #
        pcof = ry_optimization_wrapper(
            gamma=gamma,
            randomseed=randomseed,
            config_path=config_path,
            initial_amplitude_relative_maximum=initial_amplitude_relative_maximum,
            is_baseline_initial_pulse_used=true,
            verbose=false)
        #
        data_output = transpose([pcof; [gamma]])
        #
        open(data_path, "a") do io
            writedlm(io, data_output, ',');
        end
    end
end

Ipopt parameters: max # iterations = 100
Ipopt parameters: max history L-BFGS = 10
Ipopt parameters: tol = 1.0e-8
Ipopt parameters: atol = 1.0e-8
Ipopt parameters: accept # iter = 15
*** Starting the optimization ***

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       20
                     variables with only lower bounds:        0
    

Excessive output truncated after 524383 bytes.


*** Starting the optimization ***

Generate test data:

In [5]:
@time for randomseed in 1:1

    data_path = get_data_path(prefix_test, config_path, randomseed, false);
    open(data_path, "w") do io
        write(io, "");
    end;

    for gamma in full_range_test
        #
        pcof = ry_optimization_wrapper(
            gamma=gamma,
            randomseed=randomseed,
            config_path=config_path,
            initial_amplitude_relative_maximum=initial_amplitude_relative_maximum,
            is_baseline_initial_pulse_used=true,
            verbose=false)
        #
        data_output = transpose([pcof; [gamma]])
        #
        open(data_path, "a") do io
            writedlm(io, data_output, ',');
        end
    end
end

Ipopt parameters: max # iterations = 100
Ipopt parameters: max history L-BFGS = 10
Ipopt parameters: tol = 1.0e-8
Ipopt parameters: atol = 1.0e-8
Ipopt parameters: accept # iter = 15
*** Starting the optimization ***
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       20
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       20
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
       

Excessive output truncated after 524410 bytes.

-7.947034523404875e-8 < threshold = 0.0
   6 -7.9470345e-08 0.00e+00 2.50e-03  -8.6 2.21e-05    -  1.00e+00 1.00e+00f  1

Number of Iterations....: 6

                                   (scaled)                 (unscaled)
Objective...............:  -7.9470345234048746e-08   -7.9470345234048746e-08
Dual infeasibility......:   2.5015900799951578e-03    2.5015900799951578e-03
Constraint violation....:   0.0000000000000000e+00    0.0000000000000000e+00
Variable bound violation:   0.0000000000000000e+00    0.0000000000000000e+00
Complementarity.........:   3.0072804715341961e-09    3.0072804715341961e-09
Overall NLP error.......:   2.5015900799951578e-03    2.5015900799951578e-03


Number of objective function evaluations             = 13
Number of objective gradient evaluations             = 7
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 0
Number of inequality constra